In [1]:
class ArcStandard:
    def __init__(self, sentence):

        # sentence is the input for which we want to build our Arc-Standard
        self.sentence = sentence

        # here we create the buffer having an array of indexes with the same length as the sentence
        # basically, each word has its own index in this buffer
        # we have initialized the buffer having all the words in the sentence

        self.buffer = [i for i in range(len(self.sentence))] 

        # initialize the stack empty 
        
        self.stack = []

        # representation of the tree
        # every word will have a -1 assigned -> no father has been assigned yet

        self.arcs = [-1 for _ in range(len(self.sentence))]

        # three shift moves to initialize the stack

        # means that in the stack now is the ROOT
        # self.shift() it calls a method that implements this operation; we will look at it after 

        self.shift() 

        # means that in the stack now is the ROOT and He; sigma1 is He and sigma2 is the ROOT

        self.shift() 

        # until now we cannnot attach He to ROOT, if we cannot do that we add another element to the stack;
        # so, perform another shift
        # now, in the stack we have ROOT, He, began
        if len(self.sentence) > 2:
          self.shift()

    def shift(self):
       
     b1 = self.buffer[0]
     self.buffer = self.buffer[1:]
     self.stack.append(b1)
    
    def left_arc(self): 

     o1 = self.stack.pop()
     o2 = self.stack.pop()
     self.arcs[o2] = o1
     self.stack.append(o1)

     if len(self.stack) < 2 and len(self.buffer) > 0:
        self.shift()
    

    def right_arc(self):
     o1 = self.stack.pop()
     o2 = self.stack.pop()
     self.arcs[o1] = o2
     self.stack.append(o2)

     if len(self.stack) <2 and len(self.buffer) > 0:
        self.shift()
    
    def is_tree_final(self):
     return len(self.stack) == 1 and len(self.buffer) == 0
    

    def print_configuration(self):

      s = [self.sentence[i] for i in self.stack]
      b = [self.sentence[i] for i in self.buffer]
      print(s,b)
      print(self.arcs)
      

In [2]:
sentence = ["<ROOT>", "He","began","to","write","again","."]
gold = [-1, 2, 0, 4, 2, 4, 2]

parser = ArcStandard(sentence)
parser.print_configuration()

['<ROOT>', 'He', 'began'] ['to', 'write', 'again', '.']
[-1, -1, -1, -1, -1, -1, -1]


In [3]:
parser.left_arc()
parser.print_configuration()

['<ROOT>', 'began'] ['to', 'write', 'again', '.']
[-1, 2, -1, -1, -1, -1, -1]


In [4]:
parser.shift()
parser.print_configuration()

['<ROOT>', 'began', 'to'] ['write', 'again', '.']
[-1, 2, -1, -1, -1, -1, -1]


In [5]:
parser.right_arc()
parser.print_configuration()

# we are asking the parser to do this action of the right_arc; "to" has as a paretn "write" and "began"

['<ROOT>', 'began'] ['write', 'again', '.']
[-1, 2, -1, 2, -1, -1, -1]


In [6]:
class Oracle:
 def __init__(self, parser, gold_tree):
  self.parser = parser
  self.gold = gold_tree

 def is_left_arc_gold(self):
  
  # we can do the left arc if sigma2 is the children of the sigma1
  # this means that u have assigned to sigma2 all of its children and is ok to assing its parent -> static oracle
  
  # here we get our sigma1 and sigma2

  o1 = self.parser.stack[len(self.parser.stack)-1]
  o2 = self.parser.stack[len(self.parser.stack)-2]


  if self.gold[o2] == o1:
   return True
  return False

 def is_shift_gold(self):
 
  if len(self.parser.buffer) == 0:
   return False
 
  if (self.is_left_arc_gold() or self.is_right_arc_gold()):
   return False
 
  return True

 def is_right_arc_gold(self):
   o1 = self.parser.stack[len(self.parser.stack)-1]
   o2 = self.parser.stack[len(self.parser.stack)-2]

   if self.gold[o1] != o2:
    return False
   
   # we need to check that no children of sigma1 are in the rest of the buffer
   
   for i in self.parser.buffer:
    if self.gold[i] == o1:
     return False
   return True

In [7]:
sentence = ["<ROOT>", "He","began","to","write","again","."]
gold = [-1, 2, 0, 4, 2, 4, 2]

parser = ArcStandard(sentence)
oracle = Oracle(parser, gold)

parser.print_configuration()

['<ROOT>', 'He', 'began'] ['to', 'write', 'again', '.']
[-1, -1, -1, -1, -1, -1, -1]


In [8]:
print("Left Arc: " ,oracle.is_left_arc_gold())
print("Right Arc: ",oracle.is_right_arc_gold())
print("Shift: ",oracle.is_shift_gold())


Left Arc:  True
Right Arc:  False
Shift:  False


In [9]:
# oracle tells us that hte next move is the left_arc wqe do it and ask again the Oracle
parser.left_arc()
parser.print_configuration()

['<ROOT>', 'began'] ['to', 'write', 'again', '.']
[-1, 2, -1, -1, -1, -1, -1]


In [10]:
print("Left Arc: " ,oracle.is_left_arc_gold())
print("Right Arc: ",oracle.is_right_arc_gold())
print("Shift: ",oracle.is_shift_gold())

Left Arc:  False
Right Arc:  False
Shift:  True


In [11]:
while not parser.is_tree_final():
    if oracle.is_shift_gold():
        parser.shift()
    elif oracle.is_left_arc_gold():
        parser.left_arc()
    elif oracle.is_right_arc_gold():
        parser.right_arc()
        
print(parser.arcs)
print(gold)
    

[-1, 2, 0, 4, 2, 4, 2]
[-1, 2, 0, 4, 2, 4, 2]
